## Article Scraping

In [5]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os
import pymongo


In [6]:
pip install splinter

You should consider upgrading via the '/Users/harrisonpreston/opt/anaconda3/envs/PythonData/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path)

In [8]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [9]:
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [10]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8724/nasa-ula-launch-mars-2020-perseverance-rover-mission-to-red-planet/" target="_self">NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet</a></div>

In [11]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

'NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet'

In [12]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. "

## Featured Imagines

In [13]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [14]:
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [15]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [16]:
# Parse the resulting html with soup
html = browser.html
img_soup = BeautifulSoup(html, 'html.parser')

In [24]:
# Find the relative image url
img_url_rel = img_soup.select_one('figure.lede a img')
img_url_rel

In [25]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.govNone'

In [26]:
df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [27]:
# Convert DataFrame back into HTML-ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

# Mars Weather (splinter)

Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.


In [28]:
# Visit the latest Mars weather tweet from the page

twitter_weather_url = 'https://twitter.com/marswxreport?lang=en'
result = requests.get(twitter_weather_url)

In [30]:
# Scrape the latest Mars weather tweet from the page

# Create HTML object
twitter_weather_html = result.text

# Create BeautifulSoup object; parse with 'html.parser'
twitter_weather_soup = BeautifulSoup(twitter_weather_html, 'html.parser')

In [31]:
#Save the tweet text for the weather report as a variable called mars_weather.
twitter_mars_weather = twitter_weather_soup.find(class_='tweet-text').get_text()
twitter_mars_weather

AttributeError: 'NoneType' object has no attribute 'get_text'

# Mars Hemisphers (splinter)

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar’s hemispheres.

You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.

Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [32]:
# Visit https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)
print(hemi_url)

https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars


In [34]:
# Create HTML object
hemi_html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(hemi_html, 'html.parser')

In [37]:
# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys img_url and title.
# Append the dictionary with the image url string and the hemisphere title to a list. 
# This list will contain one dictionary for each hemisphere.

# Create HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retrieve items that contain hemisphere info
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls
hemisphere_image_urls = []

# Define anchor url 
hemisphere_anchor_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link appendage for full image link
    append_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit full image website 
    browser.visit(hemisphere_anchor_url + append_img_url)
    
    # Create HTML object of individual hemisphere information website 
    append_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup(append_img_html, 'html.parser')
    
    # Build full image url 
    img_url = hemisphere_anchor_url + soup.find('img', class_='wide-image')['src']
    
    # Append to a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[]